In [1]:
import random
import pandas as pd
import numpy as np
import os
import holidays

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from category_encoders.target_encoder import TargetEncoder


import warnings
warnings.filterwarnings(action='ignore') 
korea_holidays = holidays.SouthKorea(years=2023)
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

In [41]:
train = pd.read_csv('./open/train.csv')
test = pd.read_csv('./open/test.csv')
m1 = pd.read_csv('./open/external_open/대구 CCTV 정보.csv',encoding='cp949')
m2 = pd.read_csv('./open/external_open/대구 보안등 정보.csv',encoding='cp949')
m3 = pd.read_csv('./open/external_open/대구 어린이 보호 구역 정보.csv',encoding='cp949')
m4 = pd.read_csv('./open/external_open/대구 주차장 정보.csv',encoding='cp949')
train[['사망자수','중상자수','경상자수','부상자수','ECLO']]

,사망자수,중상자수,경상자수,부상자수,ECLO
0,0,1,0,0,5
1,0,0,1,0,3
2,0,0,1,0,3
3,0,1,0,0,5
4,0,0,1,0,3
...,...,...,...,...,...
39604,0,0,1,0,3
39605,0,0,1,0,3
39606,0,2,0,0,10
39607,0,0,1,0,3


In [39]:
# cols = list(test.columns)
# cols.append('ECLO')
# train = train[cols]
# print(train.info())
# train

- regression or sequential and featrue engieneering

In [42]:
print(train['사망자수'].value_counts())
print(train['중상자수'].value_counts())
print(train['경상자수'].value_counts())
print(train['부상자수'].value_counts())


0    39309
1      292
2        8
Name: 사망자수, dtype: int64
0    30136
1     8724
2      618
3       98
4       28
5        4
6        1
Name: 중상자수, dtype: int64
1     20913
0     10284
2      5560
3      1728
4       732
5       259
6        77
7        32
8         7
9         7
10        2
15        2
14        1
11        1
12        1
18        1
13        1
22        1
Name: 경상자수, dtype: int64
0     35168
1      4046
2       279
3        67
4        35
5         9
6         3
7         1
10        1
Name: 부상자수, dtype: int64


In [246]:
light_df = pd.read_csv('./open/external_open/대구 보안등 정보.csv', encoding='cp949')[['설치개수', '소재지지번주소']]
location_pattern = r'(\S+) (\S+) (\S+) (\S+)'
light_df[['도시', '구', '동', '번지']] = light_df['소재지지번주소'].str.extract(location_pattern)
light_df = light_df.drop(columns=['소재지지번주소', '번지'])
light_df = light_df.groupby(['도시', '구', '동']).sum().reset_index()
light_df.reset_index(inplace=True, drop=True)

child_area_df = pd.read_csv('./open/external_open/대구 어린이 보호 구역 정보.csv', encoding='cp949').drop_duplicates()# [['소재지지번주소']]
child_area_df['시설수'] = 1
location_pattern = r'(\S+) (\S+) (\S+) (\S+)'
child_area_df[['도시', '구', '동', '번지']] = child_area_df['소재지지번주소'].str.extract(location_pattern)
child_area_df = child_area_df.drop(columns=['소재지지번주소', '번지','위도','경도'])
child_area_df = child_area_df.groupby(['도시', '구', '동']).sum().reset_index()
child_area_df.reset_index(inplace=True, drop=True)

parking_df = pd.read_csv('./open/external_open/대구 주차장 정보.csv', encoding='cp949')
parking_df = pd.get_dummies(parking_df, columns=['급지구분'])
location_pattern = r'(\S+) (\S+) (\S+) (\S+)'
parking_df[['도시', '구', '동', '번지']] = parking_df['소재지지번주소'].str.extract(location_pattern)
parking_df = parking_df.drop(columns=['소재지지번주소', '번지','위도','경도'])
parking_df = parking_df.groupby(['도시', '구', '동']).mean().reset_index()
parking_df.reset_index(inplace=True, drop=True)
parking_df = parking_df.fillna(0)

train = pd.read_csv('./open/train.csv')
test = pd.read_csv('./open/test.csv')
cols = list(test.columns)
cols.append('ECLO')
train = train[cols]

time_pattern = r'(\d{4})-(\d{1,2})-(\d{1,2}) (\d{1,2})' 
train[['연', '월', '일', '시간']] = train['사고일시'].str.extract(time_pattern)
train[['연', '월', '일', '시간']] = train[['연', '월', '일', '시간']].apply(pd.to_numeric) 
train = train.drop(columns=['사고일시'])
test[['연', '월', '일', '시간']] = test['사고일시'].str.extract(time_pattern)
test[['연', '월', '일', '시간']] = test[['연', '월', '일', '시간']].apply(pd.to_numeric)
test = test.drop(columns=['사고일시'])

location_pattern = r'(\S+) (\S+) (\S+)'
train[['도시', '구', '동']] = train['시군구'].str.extract(location_pattern)
train = train.drop(columns=['시군구'])
test[['도시', '구', '동']] = test['시군구'].str.extract(location_pattern)
test = test.drop(columns=['시군구'])

road_pattern = r'(.+) - (.+)'
train[['도로형태1', '도로형태2']] = train['도로형태'].str.extract(road_pattern)
train = train.drop(columns=['도로형태'])

test[['도로형태1', '도로형태2']] = test['도로형태'].str.extract(road_pattern)
test = test.drop(columns=['도로형태'])

# train = pd.merge(train, light_df, how='left', on=['도시', '구', '동'])
# train = pd.merge(train, child_area_df, how='left', on=['도시', '구', '동'])
# train = pd.merge(train, parking_df, how='left', on=['도시', '구', '동'])

# test = pd.merge(test, light_df, how='left', on=['도시', '구', '동'])
# test = pd.merge(test, child_area_df, how='left', on=['도시', '구', '동'])
# test = pd.merge(test, parking_df, how='left', on=['도시', '구', '동'])

train = train.drop('ID',axis=1)
test = test.drop('ID',axis=1)
train = train.fillna(0)
test = test.fillna(0)

# train['사망자수'] = train['사망자수'].replace(2,1)

In [101]:
# from sklearn.preprocessing import LabelEncoder
# from category_encoders.target_encoder import TargetEncoder

# # test_x = test.copy()
# # train_x = train[test.columns].copy()
# # train_y = train['ECLO'].copy()
# categorical_features = list(train.dtypes[train.dtypes == "object"].index)
# for i in categorical_features:
#     le = LabelEncoder()
#     train[i] = le.fit_transform(train[i])
#     test[i] = le.transform(test[i])

In [247]:
from sklearn.ensemble import IsolationForest
from pycaret.regression import *
setup(data=train,target='ECLO'
                     ,train_size=0.9
                     ,fold=10
                     ,fold_strategy='kfold'
                     ,data_split_shuffle=True
                     ,session_id=42)

,Description,Value
0,Session id,42
1,Target,ECLO
2,Target type,Regression
3,Original data shape,"(39609, 14)"
4,Transformed data shape,"(39609, 52)"
5,Transformed train set shape,"(35648, 52)"
6,Transformed test set shape,"(3961, 52)"
7,Numeric features,4
8,Categorical features,9
9,Preprocess,True


In [249]:
compare_models(sort='RMSLE')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
huber,Huber Regressor,2.0792,10.5867,3.2523,-0.0160,0.4483,0.5313,0.5550
gbr,Gradient Boosting Regressor,2.1536,10.1449,3.1838,0.0263,0.4601,0.6244,1.2290
br,Bayesian Ridge,2.1556,10.1381,3.1827,0.0269,0.4611,0.6255,0.1660
lr,Linear Regression,2.1564,10.1422,3.1834,0.0266,0.4613,0.6255,0.1660
ridge,Ridge Regression,2.1564,10.1420,3.1833,0.0266,0.4613,0.6255,0.1570
lightgbm,Light Gradient Boosting Machine,2.1642,10.2181,3.1952,0.0193,0.4621,0.6261,0.2040
omp,Orthogonal Matching Pursuit,2.1653,10.2111,3.1942,0.0199,0.4625,0.6278,0.1550
catboost,CatBoost Regressor,2.1751,10.3370,3.2140,0.0076,0.4649,0.6277,2.0490
en,Elastic Net,2.1739,10.4226,3.2270,-0.0002,0.4676,0.6359,0.1590
lasso,Lasso Regression,2.1739,10.4226,3.2270,-0.0002,0.4676,0.6359,0.1510


HuberRegressor(alpha=0.0001, epsilon=1.35, fit_intercept=True, max_iter=100,
               tol=1e-05, warm_start=False)

In [250]:
tune_model(best[0])

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.1371,10.3555,3.2180,0.0041,0.4535,0.5627
1,2.1206,10.7925,3.2852,0.0128,0.4571,0.5865
2,2.1115,9.5851,3.0960,0.0126,0.4588,0.6004
3,2.1411,10.8747,3.2977,0.0086,0.4536,0.5611
4,2.1191,10.8210,3.2895,0.0130,0.4522,0.5608
5,2.0955,9.9035,3.1470,0.0086,0.4496,0.5651
6,2.0637,11.2476,3.3537,0.0108,0.4388,0.5319
7,2.0358,9.1802,3.0299,0.0142,0.4372,0.5461
8,2.0601,9.8961,3.1458,0.0140,0.4452,0.5617


Fitting 10 folds for each of 10 candidates, totalling 100 fits


HuberRegressor(alpha=0.1, epsilon=1.9, fit_intercept=True, max_iter=100,
               tol=1e-05, warm_start=False)

In [251]:
sample_submission = pd.read_csv('./open/sample_submission.csv')
# tp = test.iloc[:,selected_features].astype('float32')
sample_submission["ECLO"] = best_tune[0].predict(tp) #  + 0.33*best_tune[1].predict(tp) + 0.33*best_tune[2].predict(tp)
sample_submission.to_csv("huber-0.4496.csv", index=False)

---

In [35]:
light_df = pd.read_csv('./open/external_open/대구 보안등 정보.csv', encoding='cp949')[['설치개수', '소재지지번주소']]
location_pattern = r'(\S+) (\S+) (\S+) (\S+)'
light_df[['도시', '구', '동', '번지']] = light_df['소재지지번주소'].str.extract(location_pattern)
light_df = light_df.drop(columns=['소재지지번주소', '번지'])
light_df = light_df.groupby(['도시', '구', '동']).sum().reset_index()
light_df.reset_index(inplace=True, drop=True)

child_area_df = pd.read_csv('./open/external_open/대구 어린이 보호 구역 정보.csv', encoding='cp949').drop_duplicates()# [['소재지지번주소']]
child_area_df['시설수'] = 1
location_pattern = r'(\S+) (\S+) (\S+) (\S+)'
child_area_df[['도시', '구', '동', '번지']] = child_area_df['소재지지번주소'].str.extract(location_pattern)
child_area_df = child_area_df.drop(columns=['소재지지번주소', '번지','위도','경도'])
child_area_df = child_area_df.groupby(['도시', '구', '동']).sum().reset_index()
child_area_df.reset_index(inplace=True, drop=True)

parking_df = pd.read_csv('./open/external_open/대구 주차장 정보.csv', encoding='cp949')
parking_df = pd.get_dummies(parking_df, columns=['급지구분'])
location_pattern = r'(\S+) (\S+) (\S+) (\S+)'
parking_df[['도시', '구', '동', '번지']] = parking_df['소재지지번주소'].str.extract(location_pattern)
parking_df = parking_df.drop(columns=['소재지지번주소', '번지','위도','경도'])
parking_df = parking_df.groupby(['도시', '구', '동']).mean().reset_index()
parking_df.reset_index(inplace=True, drop=True)
parking_df = parking_df.fillna(0)

train = pd.read_csv('./open/train.csv')
test = pd.read_csv('./open/test.csv')
cols = list(test.columns)
cols.append('ECLO')
train = train[cols]

time_pattern = r'(\d{4})-(\d{1,2})-(\d{1,2}) (\d{1,2})' 
train[['연', '월', '일', '시간']] = train['사고일시'].str.extract(time_pattern)
train[['연', '월', '일', '시간']] = train[['연', '월', '일', '시간']].apply(pd.to_numeric) 
train = train.drop(columns=['사고일시'])
test[['연', '월', '일', '시간']] = test['사고일시'].str.extract(time_pattern)
test[['연', '월', '일', '시간']] = test[['연', '월', '일', '시간']].apply(pd.to_numeric)
test = test.drop(columns=['사고일시'])

location_pattern = r'(\S+) (\S+) (\S+)'
train[['도시', '구', '동']] = train['시군구'].str.extract(location_pattern)
train = train.drop(columns=['시군구'])
test[['도시', '구', '동']] = test['시군구'].str.extract(location_pattern)
test = test.drop(columns=['시군구'])

road_pattern = r'(.+) - (.+)'
train[['도로형태1', '도로형태2']] = train['도로형태'].str.extract(road_pattern)
train = train.drop(columns=['도로형태'])

test[['도로형태1', '도로형태2']] = test['도로형태'].str.extract(road_pattern)
test = test.drop(columns=['도로형태'])

train = pd.merge(train, light_df, how='left', on=['도시', '구', '동'])
train = pd.merge(train, child_area_df, how='left', on=['도시', '구', '동'])
# train = pd.merge(train, parking_df, how='left', on=['도시', '구', '동'])

test = pd.merge(test, light_df, how='left', on=['도시', '구', '동'])
test = pd.merge(test, child_area_df, how='left', on=['도시', '구', '동'])
# test = pd.merge(test, parking_df, how='left', on=['도시', '구', '동'])

train = train.drop('ID',axis=1)
test = test.drop('ID',axis=1)
# train = train.dropna()

train = train.fillna(0)
test = test.fillna(0)

In [36]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39609 entries, 0 to 39608
Data columns (total 17 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   요일        39609 non-null  object 
 1   기상상태      39609 non-null  object 
 2   노면상태      39609 non-null  object 
 3   사고유형      39609 non-null  object 
 4   ECLO      39609 non-null  int64  
 5   연         39609 non-null  int64  
 6   월         39609 non-null  int64  
 7   일         39609 non-null  int64  
 8   시간        39609 non-null  int64  
 9   도시        39609 non-null  object 
 10  구         39609 non-null  object 
 11  동         39609 non-null  object 
 12  도로형태1     39609 non-null  object 
 13  도로형태2     39609 non-null  object 
 14  설치개수      39609 non-null  float64
 15  CCTV설치대수  39609 non-null  float64
 16  시설수       39609 non-null  float64
dtypes: float64(3), int64(5), object(9)
memory usage: 5.4+ MB


In [37]:
# test_x = test.copy()
# train_x = train[test.columns].copy()
# train_y = train['ECLO'].copy()
categorical_features = list(train.dtypes[train.dtypes == "object"].index)
for i in categorical_features:
    le = LabelEncoder()
    train[i] = le.fit_transform(train[i])
    test[i] = le.transform(test[i])
    

In [24]:
import tensorflow as tf

def rmsle(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    
    squared_error = tf.square(tf.math.log1p(y_pred) - tf.math.log1p(y_true))

    return tf.sqrt(tf.reduce_mean(squared_error))

def loss_fn(y_true, y_pred):
    return rmsle(y_true, y_pred)

def metric_fn(y_true, y_pred):
    return rmsle(y_true, y_pred)

In [38]:
from pycaret.regression import *
# 모델 비교
setup(data=train,target='ECLO'
                     ,train_size=0.9
                     ,fold=5
                     ,fold_strategy='kfold'
                     ,data_split_shuffle=True
                     ,session_id=42
                     ,feature_selection=True
                     ,feature_selection_estimator='catboost'
                     ,n_features_to_select=0.3
                     ,verbose=200)
compare_models(sort='RMSLE')

setup(data=train,target='ECLO'
                     ,train_size=0.9
                     ,fold=5
                     ,fold_strategy='kfold'
                     ,data_split_shuffle=True
                     ,session_id=42
                     ,feature_selection=True
                     ,feature_selection_estimator='lightgbm'
                     ,n_features_to_select=0.3
                     ,verbose=200)
compare_models(sort='RMSLE')

setup(data=train,target='ECLO'
                     ,train_size=0.9
                     ,fold=5
                     ,fold_strategy='kfold'
                     ,data_split_shuffle=True
                     ,session_id=42
                     ,feature_selection=True
                     ,feature_selection_estimator='xgboost'
                     ,n_features_to_select=0.3
                     ,verbose=200)
compare_models(sort='RMSLE')

Learning rate set to 0.072011
0:	learn: 3.2226187	total: 8.05ms	remaining: 8.04s
1:	learn: 3.2181024	total: 14.4ms	remaining: 7.18s
2:	learn: 3.2139294	total: 19.9ms	remaining: 6.6s
3:	learn: 3.2102646	total: 24.1ms	remaining: 6.01s
4:	learn: 3.2066875	total: 28ms	remaining: 5.57s
5:	learn: 3.2040709	total: 32.5ms	remaining: 5.39s
6:	learn: 3.2018410	total: 37.1ms	remaining: 5.26s
7:	learn: 3.1996179	total: 40.8ms	remaining: 5.06s
8:	learn: 3.1974867	total: 45.2ms	remaining: 4.98s
9:	learn: 3.1957685	total: 50.7ms	remaining: 5.02s
10:	learn: 3.1939938	total: 55.3ms	remaining: 4.97s
11:	learn: 3.1925818	total: 59.4ms	remaining: 4.89s
12:	learn: 3.1908080	total: 63.5ms	remaining: 4.82s
13:	learn: 3.1896262	total: 68.5ms	remaining: 4.82s
14:	learn: 3.1885496	total: 72.8ms	remaining: 4.78s
15:	learn: 3.1874582	total: 77.1ms	remaining: 4.74s
16:	learn: 3.1866938	total: 81.4ms	remaining: 4.71s
17:	learn: 3.1858530	total: 85.9ms	remaining: 4.69s
18:	learn: 3.1849079	total: 90.7ms	remaining: 4

191:	learn: 3.1099572	total: 795ms	remaining: 3.34s
192:	learn: 3.1094178	total: 799ms	remaining: 3.34s
193:	learn: 3.1090063	total: 803ms	remaining: 3.33s
194:	learn: 3.1084286	total: 807ms	remaining: 3.33s
195:	learn: 3.1080457	total: 811ms	remaining: 3.33s
196:	learn: 3.1077517	total: 815ms	remaining: 3.32s
197:	learn: 3.1075265	total: 818ms	remaining: 3.31s
198:	learn: 3.1071546	total: 822ms	remaining: 3.31s
199:	learn: 3.1067159	total: 826ms	remaining: 3.3s
200:	learn: 3.1063470	total: 830ms	remaining: 3.3s
201:	learn: 3.1060172	total: 834ms	remaining: 3.29s
202:	learn: 3.1056878	total: 838ms	remaining: 3.29s
203:	learn: 3.1052746	total: 842ms	remaining: 3.28s
204:	learn: 3.1048438	total: 846ms	remaining: 3.28s
205:	learn: 3.1043789	total: 850ms	remaining: 3.27s
206:	learn: 3.1039403	total: 853ms	remaining: 3.27s
207:	learn: 3.1034193	total: 857ms	remaining: 3.26s
208:	learn: 3.1030103	total: 861ms	remaining: 3.26s
209:	learn: 3.1025358	total: 865ms	remaining: 3.25s
210:	learn: 3.

391:	learn: 3.0385147	total: 1.59s	remaining: 2.47s
392:	learn: 3.0381846	total: 1.6s	remaining: 2.47s
393:	learn: 3.0376921	total: 1.6s	remaining: 2.46s
394:	learn: 3.0375123	total: 1.61s	remaining: 2.46s
395:	learn: 3.0371086	total: 1.61s	remaining: 2.46s
396:	learn: 3.0367606	total: 1.61s	remaining: 2.45s
397:	learn: 3.0364987	total: 1.62s	remaining: 2.45s
398:	learn: 3.0361258	total: 1.62s	remaining: 2.44s
399:	learn: 3.0358591	total: 1.63s	remaining: 2.44s
400:	learn: 3.0354523	total: 1.63s	remaining: 2.44s
401:	learn: 3.0352403	total: 1.64s	remaining: 2.43s
402:	learn: 3.0349317	total: 1.64s	remaining: 2.43s
403:	learn: 3.0347036	total: 1.64s	remaining: 2.42s
404:	learn: 3.0344172	total: 1.65s	remaining: 2.42s
405:	learn: 3.0341125	total: 1.65s	remaining: 2.42s
406:	learn: 3.0335617	total: 1.65s	remaining: 2.41s
407:	learn: 3.0331493	total: 1.66s	remaining: 2.41s
408:	learn: 3.0329433	total: 1.66s	remaining: 2.4s
409:	learn: 3.0327914	total: 1.67s	remaining: 2.4s
410:	learn: 3.03

585:	learn: 2.9824118	total: 2.39s	remaining: 1.69s
586:	learn: 2.9823152	total: 2.39s	remaining: 1.68s
587:	learn: 2.9819228	total: 2.4s	remaining: 1.68s
588:	learn: 2.9816512	total: 2.4s	remaining: 1.67s
589:	learn: 2.9813147	total: 2.4s	remaining: 1.67s
590:	learn: 2.9810261	total: 2.41s	remaining: 1.67s
591:	learn: 2.9808065	total: 2.41s	remaining: 1.66s
592:	learn: 2.9803986	total: 2.42s	remaining: 1.66s
593:	learn: 2.9801658	total: 2.42s	remaining: 1.65s
594:	learn: 2.9796478	total: 2.42s	remaining: 1.65s
595:	learn: 2.9794631	total: 2.43s	remaining: 1.65s
596:	learn: 2.9792343	total: 2.43s	remaining: 1.64s
597:	learn: 2.9787685	total: 2.44s	remaining: 1.64s
598:	learn: 2.9784843	total: 2.44s	remaining: 1.63s
599:	learn: 2.9783526	total: 2.44s	remaining: 1.63s
600:	learn: 2.9780217	total: 2.45s	remaining: 1.63s
601:	learn: 2.9778574	total: 2.45s	remaining: 1.62s
602:	learn: 2.9775340	total: 2.46s	remaining: 1.62s
603:	learn: 2.9773807	total: 2.46s	remaining: 1.61s
604:	learn: 2.9

784:	learn: 2.9346672	total: 3.19s	remaining: 875ms
785:	learn: 2.9344603	total: 3.2s	remaining: 871ms
786:	learn: 2.9341063	total: 3.2s	remaining: 867ms
787:	learn: 2.9340113	total: 3.21s	remaining: 863ms
788:	learn: 2.9338396	total: 3.21s	remaining: 859ms
789:	learn: 2.9336196	total: 3.22s	remaining: 855ms
790:	learn: 2.9334585	total: 3.22s	remaining: 851ms
791:	learn: 2.9332524	total: 3.22s	remaining: 847ms
792:	learn: 2.9330670	total: 3.23s	remaining: 843ms
793:	learn: 2.9328906	total: 3.23s	remaining: 839ms
794:	learn: 2.9327642	total: 3.23s	remaining: 834ms
795:	learn: 2.9325905	total: 3.24s	remaining: 830ms
796:	learn: 2.9323861	total: 3.24s	remaining: 826ms
797:	learn: 2.9322053	total: 3.25s	remaining: 822ms
798:	learn: 2.9320616	total: 3.25s	remaining: 818ms
799:	learn: 2.9318167	total: 3.26s	remaining: 814ms
800:	learn: 2.9315908	total: 3.26s	remaining: 810ms
801:	learn: 2.9313542	total: 3.26s	remaining: 806ms
802:	learn: 2.9311699	total: 3.27s	remaining: 802ms
803:	learn: 2.

...

,Description,Value
0,Session id,42
1,Target,ECLO
2,Target type,Regression
3,Original data shape,"(39609, 17)"
4,Transformed data shape,"(39609, 5)"
5,Transformed train set shape,"(35648, 5)"
6,Transformed test set shape,"(3961, 5)"
7,Numeric features,16
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
huber,Huber Regressor,2.0971,10.8205,3.2894,-0.0382,0.4539,0.5345,0.1460
gbr,Gradient Boosting Regressor,2.1718,10.3782,3.2215,0.0042,0.4666,0.6348,0.3920
en,Elastic Net,2.1736,10.4203,3.2280,0.0001,0.4675,0.6358,0.0820
lasso,Lasso Regression,2.1736,10.4205,3.2280,0.0001,0.4675,0.6358,0.0920
ridge,Ridge Regression,2.1723,10.4171,3.2275,0.0004,0.4675,0.6355,0.0860
lr,Linear Regression,2.1723,10.4171,3.2275,0.0004,0.4675,0.6355,0.0900
lar,Least Angle Regression,2.1723,10.4171,3.2275,0.0004,0.4675,0.6355,0.0820
llar,Lasso Least Angle Regression,2.1736,10.4205,3.2280,0.0001,0.4675,0.6358,0.0880
br,Bayesian Ridge,2.1729,10.4175,3.2275,0.0004,0.4675,0.6357,0.0860
omp,Orthogonal Matching Pursuit,2.1737,10.4203,3.2280,0.0001,0.4675,0.6359,0.0820


,Description,Value
0,Session id,42
1,Target,ECLO
2,Target type,Regression
3,Original data shape,"(39609, 17)"
4,Transformed data shape,"(39609, 5)"
5,Transformed train set shape,"(35648, 5)"
6,Transformed test set shape,"(3961, 5)"
7,Numeric features,16
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
huber,Huber Regressor,2.0958,10.7856,3.2840,-0.0349,0.4546,0.5343,0.7180
gbr,Gradient Boosting Regressor,2.1522,10.1918,3.1924,0.0221,0.4616,0.6260,0.8720
lightgbm,Light Gradient Boosting Machine,2.1587,10.2256,3.1977,0.0188,0.4626,0.6270,0.6980
catboost,CatBoost Regressor,2.1646,10.3018,3.2096,0.0114,0.4637,0.6277,1.9520
lar,Least Angle Regression,2.1665,10.2926,3.2081,0.0124,0.4648,0.6347,0.6420
br,Bayesian Ridge,2.1664,10.2926,3.2081,0.0124,0.4648,0.6346,0.6360
lr,Linear Regression,2.1665,10.2926,3.2081,0.0124,0.4648,0.6347,0.6480
ridge,Ridge Regression,2.1664,10.2926,3.2081,0.0124,0.4648,0.6346,0.6680
omp,Orthogonal Matching Pursuit,2.1709,10.3910,3.2234,0.0030,0.4669,0.6352,0.6840
xgboost,Extreme Gradient Boosting,2.1815,10.4843,3.2378,-0.0062,0.4669,0.6310,0.9000


HuberRegressor()

In [ ]:
from pycaret.regression import *
# 모델 비교
setup(data=train,target='ECLO'
                     ,train_size=0.9
                     ,fold=5
                     ,fold_strategy='kfold'
                     ,data_split_shuffle=True
                     ,session_id=42
                     ,feature_selection=True
                     ,feature_selection_estimator='catboost'
                     ,n_features_to_select=0.4
                     ,verbose=200)
regression.compare_models(sort='RMSLE')

setup(data=train,target='ECLO'
                     ,train_size=0.9
                     ,fold=5
                     ,fold_strategy='kfold'
                     ,data_split_shuffle=True
                     ,session_id=42
                     ,feature_selection=True
                     ,feature_selection_estimator='lightgbm'
                     ,n_features_to_select=0.4
                     ,verbose=200)
regression.compare_models(sort='RMSLE')

setup(data=train,target='ECLO'
                     ,train_size=0.9
                     ,fold=5
                     ,fold_strategy='kfold'
                     ,data_split_shuffle=True
                     ,session_id=42
                     ,feature_selection=True
                     ,feature_selection_estimator='xgboost'
                     ,n_features_to_select=0.4
                     ,verbose=200)
regression.compare_models(sort='RMSLE')

In [ ]:
from pycaret.regression import *
# 모델 비교
setup(data=train,target='ECLO'
                     ,train_size=0.9
                     ,fold=5
                     ,fold_strategy='kfold'
                     ,data_split_shuffle=True
                     ,session_id=42
                     ,feature_selection=True
                     ,feature_selection_estimator='catboost'
                     ,n_features_to_select=0.5
                     ,verbose=200)
regression.compare_models(sort='RMSLE')

setup(data=train,target='ECLO'
                     ,train_size=0.9
                     ,fold=5
                     ,fold_strategy='kfold'
                     ,data_split_shuffle=True
                     ,session_id=42
                     ,feature_selection=True
                     ,feature_selection_estimator='lightgbm'
                     ,n_features_to_select=0.5
                     ,verbose=200)
regression.compare_models(sort='RMSLE')

setup(data=train,target='ECLO'
                     ,train_size=0.9
                     ,fold=5
                     ,fold_strategy='kfold'
                     ,data_split_shuffle=True
                     ,session_id=42
                     ,feature_selection=True
                     ,feature_selection_estimator='xgboost'
                     ,n_features_to_select=0.5
                     ,verbose=200)
regression.compare_models(sort='RMSLE')

setup(data=train,target='ECLO'
                     ,train_size=0.9
                     ,fold=5
                     ,fold_strategy='kfold'
                     ,data_split_shuffle=True
                     ,session_id=42
                     ,feature_selection=True
                     ,feature_selection_estimator='gbr'
                     ,n_features_to_select=0.5
                     ,verbose=200)
regression.compare_models(sort='RMSLE')

In [ ]:
from pycaret.regression import *
# 모델 비교
train_size = 0.8
fold = 10
n_features_to_select = 0.4
setup(data=train,target='ECLO'
                     ,train_size=train_size
                     ,fold=fold
                     ,fold_strategy='kfold'
                     ,data_split_shuffle=True
                     ,session_id=42
                     ,feature_selection=True
                     ,feature_selection_estimator='catboost'
                     ,n_features_to_select=n_features_to_select
                     ,verbose=200)
regression.compare_models(sort='RMSLE')

setup(data=train,target='ECLO'
                     ,train_size=train_size
                     ,fold=fold
                     ,fold_strategy='kfold'
                     ,data_split_shuffle=True
                     ,session_id=42
                     ,feature_selection=True
                     ,feature_selection_estimator='lightgbm'
                     ,n_features_to_select=n_features_to_select
                     ,verbose=200)
regression.compare_models(sort='RMSLE')

setup(data=train,target='ECLO'
                     ,train_size=train_size
                     ,fold=fold
                     ,fold_strategy='kfold'
                     ,data_split_shuffle=True
                     ,session_id=42
                     ,feature_selection=True
                     ,feature_selection_estimator='xgboost'
                     ,n_features_to_select=n_features_to_select
                     ,verbose=200)
regression.compare_models(sort='RMSLE')

setup(data=train,target='ECLO'
                     ,train_size=train_size
                     ,fold=fold
                     ,fold_strategy='kfold'
                     ,data_split_shuffle=True
                     ,session_id=42
                     ,feature_selection=True
                     ,feature_selection_estimator='gbr'
                     ,n_features_to_select=n_features_to_select
                     ,verbose=200)
regression.compare_models(sort='RMSLE')

In [137]:
setup(data=train,target='ECLO'
                     ,train_size=0.8
                     ,fold=5
                     ,fold_strategy='kfold'
                     ,data_split_shuffle=True
                     ,session_id=42
                     ,feature_selection=True
                     ,feature_selection_estimator='gbr'
                     ,n_features_to_select=0.5
                     ,verbose=200)
best = regression.compare_models(sort='RMSLE',n_select=3)
# best_tune = [tune_model(i) for i in best]

,Description,Value
0,Session id,42
1,Target,ECLO
2,Target type,Regression
3,Original data shape,"(39609, 28)"
4,Transformed data shape,"(39609, 14)"
5,Transformed train set shape,"(31687, 14)"
6,Transformed test set shape,"(7922, 14)"
7,Numeric features,27
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
huber,Huber Regressor,2.0702,10.8633,3.2953,-0.0476,0.4527,0.5148,1.0460
gbr,Gradient Boosting Regressor,2.1390,10.0585,3.1710,0.0298,0.4586,0.6222,1.5020
lightgbm,Light Gradient Boosting Machine,2.1554,10.1530,3.1858,0.0208,0.4614,0.6258,1.0360
ridge,Ridge Regression,2.1573,10.2125,3.1952,0.0150,0.4632,0.6318,0.9980
lar,Least Angle Regression,2.1573,10.2125,3.1952,0.0150,0.4632,0.6318,0.9480
br,Bayesian Ridge,2.1569,10.2127,3.1952,0.0150,0.4632,0.6316,0.9300
lr,Linear Regression,2.1573,10.2125,3.1952,0.0150,0.4632,0.6318,0.9560
catboost,CatBoost Regressor,2.1731,10.3419,3.2153,0.0025,0.4661,0.6286,2.4340
en,Elastic Net,2.1634,10.3570,3.2176,0.0011,0.4661,0.6327,0.9440
llar,Lasso Least Angle Regression,2.1635,10.3572,3.2177,0.0011,0.4661,0.6328,0.9400


In [138]:
best_tune = [tune_model(i) for i in best]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.1005,10.1169,3.1807,-0.0115,0.4603,0.5815
1,2.1425,11.3124,3.3634,-0.0223,0.4606,0.5541
2,2.0678,10.2833,3.2068,-0.0299,0.4518,0.5375
3,2.0717,10.8879,3.2997,-0.0371,0.4513,0.5280
4,2.0773,10.6170,3.2584,-0.0317,0.4570,0.5544
Mean,2.0920,10.6435,3.2618,-0.0265,0.4562,0.5511
Std,0.0277,0.4277,0.0654,0.0089,0.0040,0.0182


Fitting 5 folds for each of 10 candidates, totalling 50 fits


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.1651,9.7146,3.1168,0.0288,0.4692,0.6588
1,2.1769,10.6655,3.2658,0.0362,0.4603,0.6142
2,2.1179,9.6585,3.1078,0.0327,0.4549,0.6119
3,2.1126,10.1992,3.1936,0.0285,0.4517,0.5996
4,2.1256,10.0262,3.1664,0.0257,0.4584,0.6266
Mean,2.1396,10.0528,3.1701,0.0304,0.4589,0.6222
Std,0.0262,0.3653,0.0573,0.0037,0.0059,0.0202


Fitting 5 folds for each of 10 candidates, totalling 50 fits


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.1618,9.7123,3.1165,0.0290,0.4678,0.6562
1,2.1777,10.6638,3.2655,0.0364,0.4601,0.6142
2,2.1223,9.6579,3.1077,0.0327,0.4552,0.6143
3,2.1208,10.2136,3.1959,0.0271,0.4527,0.6040
4,2.1290,10.0220,3.1658,0.0261,0.4585,0.6281
Mean,2.1423,10.0539,3.1703,0.0303,0.4589,0.6234
Std,0.0231,0.3666,0.0576,0.0038,0.0052,0.0181


Fitting 5 folds for each of 10 candidates, totalling 50 fits


# submission part

In [175]:
import lightgbm as lgb

train_data = lgb.Dataset(train.drop('ECLO',axis=1), label=train['ECLO'])
params = {'objective': 'regression', 'metric': 'rmsle', 'random_state':42}
model = lgb.train(params, train_data, num_boost_round=100)
feature_importance = model.feature_importance()
sorted_feature_indices = np.argsort(feature_importance)[::-1]

ratio = 0.6
k = int(len(train.drop('ECLO',axis=1).columns)*ratio)
selected_features = sorted_feature_indices[:k]

setup(data=train.drop('ECLO',axis=1).iloc[:,selected_features],target=train['ECLO']
                     ,train_size=0.8
                     ,fold=5
                     ,fold_strategy='kfold'
                     ,data_split_shuffle=True
                     ,session_id=42
                     ,verbose=200)
best = regression.compare_models(sort='RMSLE',n_select=3)
# best_tune = [tune_model(i) for i in best]

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004982 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 843
[LightGBM] [Info] Number of data points in the train set: 39609, number of used features: 26
[LightGBM] [Info] Start training from score 4.726704


,Description,Value
0,Session id,42
1,Target,ECLO
2,Target type,Regression
3,Original data shape,"(39609, 17)"
4,Transformed data shape,"(39609, 17)"
5,Transformed train set shape,"(31687, 17)"
6,Transformed test set shape,"(7922, 17)"
7,Numeric features,16
8,Preprocess,True
9,Imputation type,simple


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,12:45:14
Status,. . . . . . . . . . . . . . . . . .,Fitting 5 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Lasso Regression


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lr,Linear Regression,2.1568,10.2026,3.1936,0.0159,0.4630,0.6313,0.7480



KeyboardInterrupt



In [173]:
best_tune = [tune_model(i) for i in best]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.1007,10.1716,3.1893,-0.0169,0.4617,0.5795
1,2.1377,11.4702,3.3868,-0.0365,0.4631,0.5413
2,2.0653,10.2765,3.2057,-0.0292,0.4502,0.5335
3,2.0681,10.8509,3.2941,-0.0336,0.4508,0.5276
4,2.0727,10.5308,3.2451,-0.0233,0.4536,0.5539
Mean,2.0889,10.6600,3.2642,-0.0279,0.4559,0.5472
Std,0.0274,0.4680,0.0712,0.0071,0.0055,0.0184


Fitting 5 folds for each of 10 candidates, totalling 50 fits


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.1650,9.7163,3.1171,0.0286,0.4690,0.6581
1,2.1786,10.6817,3.2683,0.0347,0.4606,0.6141
2,2.1167,9.6581,3.1077,0.0327,0.4550,0.6118
3,2.1124,10.1895,3.1921,0.0294,0.4517,0.5993
4,2.1218,9.9967,3.1617,0.0286,0.4577,0.6256
Mean,2.1389,10.0485,3.1694,0.0308,0.4588,0.6218
Std,0.0274,0.3705,0.0581,0.0025,0.0059,0.0200


Fitting 5 folds for each of 10 candidates, totalling 50 fits


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.1626,9.7221,3.1180,0.0280,0.4682,0.6568
1,2.1782,10.6697,3.2665,0.0358,0.4601,0.6141
2,2.1201,9.6584,3.1078,0.0327,0.4551,0.6135
3,2.1195,10.2030,3.1942,0.0281,0.4525,0.6034
4,2.1289,10.0248,3.1662,0.0258,0.4586,0.6279
Mean,2.1419,10.0556,3.1705,0.0301,0.4589,0.6231
Std,0.0240,0.3657,0.0574,0.0036,0.0054,0.0185


Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [201]:
from sklearn.inspection import permutation_importance
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train.drop('ECLO',axis=1), train['ECLO'], test_size=0.2, random_state=42)

model = GradientBoostingRegressor(random_state=42)
model.fit(X_train, y_train)

result = permutation_importance(model, X_test, y_test, n_repeats=10, random_state=42)
feature_importance = result.importances_mean

tp = {i:feature_importance[i] for i in range(len(feature_importance))}
sorted_dict = dict(sorted(tp.items(), key=lambda item: item[1], reverse=True))
sorted_dict

ratio = 0.5
k = int(len(train.drop('ECLO',axis=1).columns)*ratio)
selected_features = list(sorted_dict.keys())[:k]

setup(data=train.drop('ECLO',axis=1).iloc[:,selected_features],target=train['ECLO']
                     ,train_size=0.8
                     ,fold=10
                     ,fold_strategy='kfold'
                     ,data_split_shuffle=True
                     ,session_id=42
                     ,verbose=200)
best = regression.compare_models(sort='RMSLE',n_select=3)

,Description,Value
0,Session id,42
1,Target,ECLO
2,Target type,Regression
3,Original data shape,"(39609, 14)"
4,Transformed data shape,"(39609, 14)"
5,Transformed train set shape,"(31687, 14)"
6,Transformed test set shape,"(7922, 14)"
7,Numeric features,13
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
huber,Huber Regressor,2.0726,10.9603,3.3076,-0.0570,0.4538,0.5095,0.0870
gbr,Gradient Boosting Regressor,2.1373,10.0396,3.1658,0.0316,0.4581,0.6215,0.6190
lightgbm,Light Gradient Boosting Machine,2.1492,10.1197,3.1786,0.0237,0.4602,0.6233,0.0810
br,Bayesian Ridge,2.1571,10.1993,3.1910,0.0161,0.4628,0.6312,0.0200
lr,Linear Regression,2.1574,10.1992,3.1909,0.0161,0.4629,0.6313,0.5230
ridge,Ridge Regression,2.1574,10.1992,3.1909,0.0161,0.4629,0.6313,0.0190
lar,Least Angle Regression,2.1574,10.1992,3.1909,0.0161,0.4629,0.6313,0.0200
catboost,CatBoost Regressor,2.1635,10.2679,3.2018,0.0093,0.4641,0.6259,2.1650
en,Elastic Net,2.1616,10.3443,3.2135,0.0022,0.4658,0.6320,0.0180
lasso,Lasso Regression,2.1622,10.3477,3.2140,0.0019,0.4659,0.6322,0.0200


In [202]:
best_tune = [tune_model(i) for i in best]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.0924,10.7773,3.2829,-0.0310,0.4542,0.5441
1,2.1014,9.8447,3.1376,-0.0307,0.4619,0.5717
2,2.1396,11.3616,3.3707,-0.0376,0.4601,0.5370
3,2.1288,11.5868,3.4039,-0.0362,0.4570,0.5262
4,2.0888,10.6278,3.2600,-0.0316,0.4553,0.5422
5,2.0486,10.0150,3.1646,-0.0361,0.4407,0.5070
6,2.1363,12.5877,3.5479,-0.0431,0.4560,0.5176
7,2.0038,9.1879,3.0312,-0.0307,0.4364,0.5153
8,2.0583,10.3511,3.2173,-0.0331,0.4475,0.5295


Fitting 10 folds for each of 10 candidates, totalling 100 fits


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.1494,10.1209,3.1813,0.0318,0.4624,0.6382
1,2.1755,9.3061,3.0506,0.0257,0.4749,0.6768
2,2.1929,10.5697,3.2511,0.0347,0.4635,0.6258
3,2.1727,10.7806,3.2834,0.0359,0.4589,0.6089
4,2.1420,10.0077,3.1635,0.0285,0.4639,0.6328
5,2.0863,9.2730,3.0452,0.0406,0.4442,0.5883
6,2.1743,11.7903,3.4337,0.0230,0.4591,0.5989
7,2.0496,8.5947,2.9317,0.0358,0.4437,0.6003
8,2.1012,9.7143,3.1168,0.0304,0.4533,0.6112


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.1498,10.1002,3.1781,0.0337,0.4616,0.6380
1,2.1751,9.3018,3.0499,0.0261,0.4740,0.6763
2,2.1919,10.5288,3.2448,0.0384,0.4627,0.6262
3,2.1744,10.7915,3.2850,0.0349,0.4592,0.6101
4,2.1461,9.9811,3.1593,0.0311,0.4637,0.6341
5,2.0893,9.2739,3.0453,0.0405,0.4445,0.5898
6,2.1794,11.7795,3.4321,0.0239,0.4593,0.6015
7,2.0561,8.6453,2.9403,0.0302,0.4448,0.6025
8,2.1091,9.7496,3.1224,0.0269,0.4546,0.6150


Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [222]:
sample_submission = pd.read_csv('./open/sample_submission.csv')
tp = test.iloc[:,selected_features].astype('float32')
sample_submission["ECLO"] = 0.34*best_tune[0].predict(tp) + 0.33*best_tune[1].predict(tp) + 0.33*best_tune[2].predict(tp)
sample_submission.to_csv("submission.csv", index=False)

- 다중값 RNN, ECLO 컴포넌트 예측